# Vertex Cover with qubovert

We'll first solve a Vertex Cover problem with the `PCBO` class, and then solve it using the `qubovert.problems` library.

**Contents**

1. <a href="#PCBO">Solving Vertex Cover with `qubovert.PCBO`</a>
2. <a href="#problems">Solving Vertex Cover with the `qubovert.problems` library</a>

*qubovert* must be pip installed.

The goal of the VertexCover problem is to find the smallest number of verticies that can be colored such that every edge of the graph is incident to a colored vertex. Let's begin by making a random graph (seed random for consistent results).

In [1]:
import random
random.seed(0)

n, p = 10, 0.5

vertices = set(range(n))
edges = set((i, j) for i in range(n) for j in range(i+1, n) if random.random() <= p)
print(edges)

{(4, 7), (2, 6), (6, 9), (6, 8), (4, 5), (5, 6), (7, 9), (8, 9), (0, 6), (1, 5), (1, 8), (3, 6), (0, 4), (0, 9), (0, 3), (0, 8), (3, 4), (5, 8), (3, 5)}


<div id="PCBO" />
    
## Solving Vertex Cover with `qubovert.PCBO`

See 
    
    Andrew Lucas. Ising formulations of many np problems. Frontiers in Physics, 2:5, 2014.

for a good review on converting common NP problems to QUBOs.
    

Create a Polynomial Constrained Boolean Optimization (``PCBO``) object.

In [2]:
from qubovert import PCBO

H = PCBO()

Let each vertex be a boolean variable that is 1 if it is colored, otherwise 0. Then we want to minimize the number of colored vertices:

In [3]:
for i in vertices:
    H[(i,)] += 1
    
# another equivalent way to create H is the following

# from qubovert import boolean_var
# for i in verticies:
#     H += boolean_var(i)

Now we want to enforce the constraint that every edge is adjacent to at least one colored vertex. The `PCBO.add_constraint_OR(*variables, lam)` method makes it favorable for the OR of the variables to be 1, by penalizing those bit assignments that violate the OR clause with a positive penalty `lam`. For now, we will define a symbol `lam` that we can tune easily later to find the right value.

In [4]:
# !pip install sympy
from sympy import Symbol

lam = Symbol("lam")

for i, j in edges:
    H.add_constraint_OR(i, j, lam=lam)

In [5]:
print(H.constraints, "\n")
print("Equality constraints:\n")
for x in H.constraints.get("eq", []):
    print(x, "\n")

{'eq': [{(4,): -1, (4, 7): 1, (7,): -1, (): 1}, {(2,): -1, (2, 6): 1, (6,): -1, (): 1}, {(6,): -1, (6, 9): 1, (9,): -1, (): 1}, {(6,): -1, (6, 8): 1, (8,): -1, (): 1}, {(4,): -1, (4, 5): 1, (5,): -1, (): 1}, {(5,): -1, (5, 6): 1, (6,): -1, (): 1}, {(7,): -1, (7, 9): 1, (9,): -1, (): 1}, {(8,): -1, (8, 9): 1, (9,): -1, (): 1}, {(0,): -1, (0, 6): 1, (6,): -1, (): 1}, {(1,): -1, (1, 5): 1, (5,): -1, (): 1}, {(1,): -1, (1, 8): 1, (8,): -1, (): 1}, {(3,): -1, (3, 6): 1, (6,): -1, (): 1}, {(0,): -1, (0, 4): 1, (4,): -1, (): 1}, {(0,): -1, (0, 9): 1, (9,): -1, (): 1}, {(0,): -1, (0, 3): 1, (3,): -1, (): 1}, {(0,): -1, (0, 8): 1, (8,): -1, (): 1}, {(3,): -1, (3, 4): 1, (4,): -1, (): 1}, {(5,): -1, (5, 8): 1, (8,): -1, (): 1}, {(3,): -1, (3, 5): 1, (5,): -1, (): 1}]} 

Equality constraints:

{(4,): -1, (4, 7): 1, (7,): -1, (): 1} 

{(2,): -1, (2, 6): 1, (6,): -1, (): 1} 

{(6,): -1, (6, 9): 1, (9,): -1, (): 1} 

{(6,): -1, (6, 8): 1, (8,): -1, (): 1} 

{(4,): -1, (4, 5): 1, (5,): -1, (): 1} 

{

Notice that the constraints are automatically added to the objective function. The `'eq'` key of the constraints dictionary indicates that the quantity equals zero. Other possible keys are `'lt'`, `'le'`, `'gt'`, `'ge'`, and `'ne'`. See the docstrings for `PCBO.add_constraint_eq_zero`, `PCBO.add_constraint_lt_zero`, `PCBO.add_constraint_le_zero`, `PCBO.add_constraint_gt_zero`, `PCBO.add_constraint_ge_zero`, and `PCBO.add_constraint_ne_zero` for info.

Here is the final PUBO formulation of the problem. Finding the solution is equivalent to minimizing it as `lam` goes to infinity.

In [6]:
print(H, "\n")
print("Number of variables:", H.num_binary_variables)
print("degree:", H.degree)

{(0,): 1 - 5*lam, (1,): 1 - 2*lam, (2,): 1 - lam, (3,): 1 - 4*lam, (4,): 1 - 4*lam, (5,): 1 - 5*lam, (6,): 1 - 6*lam, (7,): 1 - 2*lam, (8,): 1 - 5*lam, (9,): 1 - 4*lam, (4, 7): lam, (): 19*lam, (2, 6): lam, (6, 9): lam, (6, 8): lam, (4, 5): lam, (5, 6): lam, (7, 9): lam, (8, 9): lam, (0, 6): lam, (1, 5): lam, (1, 8): lam, (3, 6): lam, (0, 4): lam, (0, 9): lam, (0, 3): lam, (0, 8): lam, (3, 4): lam, (5, 8): lam, (3, 5): lam} 

Number of variables: 10
degree: 2


Here we see the full PUBO representing the PCBO. Let's solve it bruteforce to make sure everything is working well. `H.solve_bruteforce` will ensure that all the constraints are satisfied.

In [7]:
H_solutions = H.solve_bruteforce(all_solutions=True)
for sol in H_solutions:
    print(sol)

{0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1, 6: 1, 7: 0, 8: 1, 9: 1}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 1}
{0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 0, 8: 1, 9: 1}
{0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1}


We see there are 6 possible solutions to this Vertex Cover problem.

Now let's solve this problem with a generic QUBO solver. Since the degree of this PCBO is 2, it is already in fact a QUBO. However, let's still do the following steps, since they work in general even when the degree is larger than 2.

In [8]:
Q = H.to_qubo()
print(Q)

{(0,): 1 - 5*lam, (1,): 1 - 2*lam, (2,): 1 - lam, (3,): 1 - 4*lam, (4,): 1 - 4*lam, (5,): 1 - 5*lam, (6,): 1 - 6*lam, (7,): 1 - 2*lam, (8,): 1 - 5*lam, (9,): 1 - 4*lam, (4, 7): lam, (2, 6): lam, (6, 9): lam, (6, 8): lam, (4, 5): lam, (5, 6): lam, (7, 9): lam, (8, 9): lam, (0, 6): lam, (1, 5): lam, (1, 8): lam, (3, 6): lam, (0, 4): lam, (0, 9): lam, (0, 3): lam, (0, 8): lam, (3, 4): lam, (5, 8): lam, (3, 5): lam, (): 19*lam}


For testing purposes, let's solve this with bruteforce to see what the proper value of `lam` should be to enforce the constraints. Notice how we remap the QUBO solution to the PCBO solution with `H.convert_solution(x)`. Also note that although the `H.solve_bruteforce` method ensured that the solutions satisfied all the constraints, the `Q.solve_bruteforce` method will not! This is because the `Q` is a QUBO object which is unconstrained, whereas the `H` is a PCBO object which is allowed to be constrained.

In [9]:
for l in (1, 2, 3):
    Q_temp = Q.subs({lam: l})
    solutions = Q_temp.solve_bruteforce(all_solutions=True)
    solutions = [H.convert_solution(x) for x in solutions]
    print('lam', l)
    for s in solutions:
        print("\t", s, "is", "valid" if H.is_solution_valid(s) else "invalid")
    print()

lam 1
	 {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 0, 8: 1, 9: 1} is invalid
	 {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 1} is invalid
	 {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1, 6: 1, 7: 0, 8: 1, 9: 1} is valid
	 {0: 0, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 1} is valid
	 {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0} is invalid
	 {0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1} is invalid
	 {0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 0, 8: 1, 9: 0} is invalid
	 {0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 0, 8: 1, 9: 1} is valid
	 {0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0} is valid
	 {0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0} is valid
	 {0: 1, 1: 1, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1} is valid

lam 2
	 {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1, 6: 1, 7: 0, 8: 1, 9: 1} is valid
	 {0: 0, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 1} is valid
	 {0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6:

We see that `lam = 2` is sufficient to enforce the constraints, and we get back the same results as from the PCBO solve. So let's update our QUBO.

In [10]:
Q_good = Q.subs({lam: 2})

Now let's solve the QUBO with D'Wave's simulated annealer.

In [11]:
#!pip install dwave-neal
from neal import SimulatedAnnealingSampler

sampler = SimulatedAnnealingSampler()

Note that their software package takes in a specific form for QUBOs, namely, the keys of the dictionary must be two element tuples. This form can be accessed from `Q` with `Q.Q` and from `Q_good` with `Q_good.Q`.

In [12]:
qubo_sample = sampler.sample_qubo(Q_good.Q, num_reads=500)
print("objective function:", qubo_sample.first.energy + Q_good.offset, "\n")

qubo_solution = qubo_sample.first.sample
print("qubo solution:", qubo_solution, "\n")

solution = H.convert_solution(qubo_solution)
print("pcbo solution:", solution)
print("objective function:", H.value(solution), "\n")

print("The solution is", "valid" if H.is_solution_valid(solution) else "invalid")

objective function: 6.0 

qubo solution: {0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0} 

pcbo solution: {0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0}
objective function: 6 

The solution is valid


Notice that `H.is_solution_valid` checks if all the constraints are satisfied. Notice also that the objective function is equal to the number of colred edges.

Now we'll solve an QUSO formulation of our problem.

In [13]:
L = H.to_quso().subs({lam: 2})
# note that we cannot do H.subs({lam: 2}).to_quso()!! This is because H.subs({lam: 2})
# creates a new PCBO object, and it's mapping from variables labels to integers may be
# different than H's mapping. For example, try H.mapping == H.subs({lam: 2}).mapping a
# few times. They will often be different.
print(L)

{(0,): 2.0, (): 14.5, (1,): 0.5, (3,): 1.5, (4,): 1.5, (5,): 2.0, (6,): 2.5, (7,): 0.5, (8,): 2.0, (9,): 1.5, (4, 7): 0.5, (2, 6): 0.5, (6, 9): 0.5, (6, 8): 0.5, (4, 5): 0.5, (5, 6): 0.5, (7, 9): 0.5, (8, 9): 0.5, (0, 6): 0.5, (1, 5): 0.5, (1, 8): 0.5, (3, 6): 0.5, (0, 4): 0.5, (0, 9): 0.5, (0, 3): 0.5, (0, 8): 0.5, (3, 4): 0.5, (5, 8): 0.5, (3, 5): 0.5}


Similar to their QUBO solver, D'Wave's QUSO solver accepts a specific form for QUSO models, namely a linear term dictionary and a quadratic term dictionary. These can be accessed with `L.h` and `L.J`.

In [14]:
quso_sample = sampler.sample_ising(L.h, L.J, num_reads=500)
print("objective function:", quso_sample.first.energy + L.offset, "\n")

quso_solution = quso_sample.first.sample
print("quso solution:", quso_solution, "\n")

solution = H.convert_solution(quso_solution)
print("pcbo solution:", solution)
print("objective function:", H.value(solution), "\n")

print("The solution is", "valid" if H.is_solution_valid(solution) else "invalid")

objective function: 6.0 

quso solution: {0: 1, 1: -1, 2: 1, 3: -1, 4: -1, 5: 1, 6: -1, 7: 1, 8: -1, 9: -1} 

pcbo solution: {0: 0, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 1}
objective function: 6 

The solution is valid


We see that the solution is again valid.

<div id="problems" />

## Solving Vertex Cover with the `qubovert.problems` library

Here we will use the ``qubovert.problems`` library to abstract away all the thinking involved in formulating the Vertex Cover problem. Begin by importing the `VertexCover` class.

In [15]:
from qubovert.problems import VertexCover

problem = VertexCover(edges)

Let's solve it bruteforce to make sure everything is working.

In [16]:
print(problem.solve_bruteforce(all_solutions=True))

[{3, 4, 5, 6, 8, 9}, {1, 3, 4, 6, 8, 9}, {0, 4, 5, 6, 8, 9}, {0, 4, 5, 6, 7, 8}, {0, 3, 5, 6, 7, 8}, {0, 1, 4, 5, 6, 9}]


Notice how the format of each solution is a set of vertices, indicating which vertices to color. Again we find 6 valid solutions as we did with the PCBO method above.

Now let's solve the QUBO with D'Wave's simulated annealer. The `problem.to_qubo` method takes in as an argument some lagrange multipliers for the QUBO formulation, but if left blank they are automatically formed to enforce the constraints.

In [17]:
Q = problem.to_qubo()

print("Number of QUBO variables:", Q.num_binary_variables, "\n")

qubo_solution = qubo_sample.first.sample
print("qubo solution:", qubo_solution)

qubo_sample = sampler.sample_qubo(Q.Q, num_reads=500)
print("qubo objective function:", qubo_sample.first.energy + Q.offset, "\n")

solution = problem.convert_solution(qubo_solution)
print("problem solution (which vertices to cover):", solution, "\n")

print("The solution is", "valid" if problem.is_solution_valid(solution) else "invalid")

Number of QUBO variables: 10 

qubo solution: {0: 1, 1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0}
qubo objective function: 6.0 

problem solution (which vertices to cover): {0, 3, 5, 6, 7, 8} 

The solution is valid


Note that the QUBO solution that maps integers 0 through 9 to values 0 or 1 may have a different mapping. Ie the QUBO label 0 may not actually correspond to vertex 0, this is why it is crucial to use the `problem.convert_solution` method!

Now let's solve the QUSO.

In [18]:
L = problem.to_quso()

print("Number of QUSO variables:", L.num_binary_variables, "\n")

quso_solution = quso_sample.first.sample
print("quso solution:", quso_solution)

quso_sample = sampler.sample_ising(L.h, L.J, num_reads=500)
print("quso objective function:", quso_sample.first.energy + L.offset, "\n")

solution = problem.convert_solution(quso_solution)
print("problem solution:", solution, "\n")

print("The solution is", "valid" if problem.is_solution_valid(solution) else "invalid")

Number of QUSO variables: 10 

quso solution: {0: 1, 1: -1, 2: 1, 3: -1, 4: -1, 5: 1, 6: -1, 7: 1, 8: -1, 9: -1}
quso objective function: 6.0 

problem solution: {1, 3, 4, 6, 8, 9} 

The solution is valid


Thus we have solved the Vertex Cover problem!